In [1]:
import pandas as pd
import numpy as np
import copy
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
df202301 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df202302 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [3]:
df202301.shape

(3066766, 19)

###### Q1 There are 19 columns

In [4]:
def get_duration(dataframe):
    dataframe['duration'] = dataframe['tpep_dropoff_datetime']-dataframe['tpep_pickup_datetime']
    dataframe['duration'] = dataframe['duration'].map(lambda x:round(x.total_seconds()/60, 2))
    return dataframe

In [5]:
df202301 = get_duration(df202301)
df202302 = get_duration(df202302)

In [6]:
np.std(df202301.duration)

42.59434623473485

##### Q2 Standard deviation of the trips duration in Janurary is 42.59

In [7]:
df = df202301[(df202301['duration']>=1) & (df202301['duration'] <= 60)]

In [8]:
df.shape[0]/df202301.shape[0]

0.9812202822125979

###### Q3 98% of the records left after dropping outliers

In [9]:
df['PULocationID'] = df['PULocationID'].map(lambda x:str(x))
df['DOLocationID'] = df['DOLocationID'].map(lambda x:str(x))

/tmp/ipykernel_13379/3729636684.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PULocationID'] = df['PULocationID'].map(lambda x:str(x))
/tmp/ipykernel_13379/3729636684.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DOLocationID'] = df['DOLocationID'].map(lambda x:str(x))


In [10]:
df_location = df[['PULocationID', 'DOLocationID']]

In [11]:
location_dict = df_location.to_dict('records')

In [17]:
vectorizer = DictVectorizer()
X = vectorizer.fit_transform(location_dict)

In [18]:
X.shape

(3009173, 515)

##### Q4 Dimension of the matrix is 515

In [21]:
y = np.array(df['duration'])

In [22]:
lr = LinearRegression().fit(X,y)

In [23]:
y_pred = lr.predict(X)
mse = mean_squared_error(y_pred, y)

In [24]:
np.sqrt(mse)

7.6492606820206674

##### Q5 RMSE on the train is 7.64

In [14]:
df2 = df202302[(df202302.duration >= 1) & (df202302.duration <= 60)]

In [15]:
df2['PULocationID'] = df2['PULocationID'].apply(lambda x:str(x))
df2['DOLocationID'] = df2['DOLocationID'].apply(lambda x:str(x))
df2_location = df2[['PULocationID', 'DOLocationID']]

/tmp/ipykernel_13379/3200427470.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['PULocationID'] = df2['PULocationID'].apply(lambda x:str(x))
/tmp/ipykernel_13379/3200427470.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['DOLocationID'] = df2['DOLocationID'].apply(lambda x:str(x))


In [19]:
loc_dict = df2_location.to_dict('records')
X_val = vectorizer.transform(loc_dict)
X_val.shape

(2855951, 515)

In [20]:
y_val = np.array(df2['duration'])

In [25]:
y_val_pred = lr.predict(X_val)
mse_val = mean_squared_error(y_val_pred, y_val)
np.sqrt(mse_val)

7.811835500559634